In [15]:
"""
Make generate images that distort the existing crab images.
These images are saved to a file and will later be used to train the model.
Distort by: color, haze, gausian noise, motion blur, brightness, roation, scale

Code by Anika and Claude, 2025, for MATE
"""

'\nMake generate images that distort the existing crab images.\nThese images are saved to a file and will later be used to train the model.\nDistort by: color, haze, gausian noise, motion blur, brightness, roation, scale\n\nCode by Anika and Claude, 2025, for MATE\n'

In [16]:
import cv2
import numpy as np
import os
from pathlib import Path

In [17]:
class makeSingularCrabImage:
    """
    Generate realistic underwater image augmentations for green crab detection.
    Based on underwater distortion characteristics from scientific literature.
    """
    
    def __init__(self, output_dir='augmented_images'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
    
    def rotate_image(self, img, angle):
        """Rotate image by specified angle with white background."""
        h, w = img.shape[:2]
        center = (w // 2, h // 2)
        
        # Calculate new bounding box size to fit rotated image
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1])
        new_w = int((h * sin) + (w * cos))
        new_h = int((h * cos) + (w * sin))
        
        # Adjust rotation matrix for new center
        M[0, 2] += (new_w / 2) - center[0]
        M[1, 2] += (new_h / 2) - center[1]
        
        # Rotate with white background
        rotated = cv2.warpAffine(img, M, (new_w, new_h), 
                                 borderMode=cv2.BORDER_CONSTANT,
                                 borderValue=(255, 255, 255))
        return rotated
    
    def scale_image(self, img, scale_factor):
        """Scale image without maintaining original dimensions."""
        h, w = img.shape[:2]
        new_h, new_w = int(h * scale_factor), int(w * scale_factor)
        
        # Resize
        scaled = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        return scaled
    
    def crop_to_content(self, img, margin=20):
        """Crop image to content with minimal white space."""
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Find non-white pixels (threshold at 250 to allow slight variations)
        _, thresh = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Get bounding box of all contours
            x_min, y_min = img.shape[1], img.shape[0]
            x_max, y_max = 0, 0
            
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x + w)
                y_max = max(y_max, y + h)
            
            # Add margin
            x_min = max(0, x_min - margin)
            y_min = max(0, y_min - margin)
            x_max = min(img.shape[1], x_max + margin)
            y_max = min(img.shape[0], y_max + margin)
            
            # Crop
            return img[y_min:y_max, x_min:x_max]
        
        return img
    
    def generate_augmented_dataset(self, image_path, num_variations=20):
        """
        Generate comprehensive augmented dataset.
        
        Args:
            image_path: path to original crab image
            num_variations: Number of augmented versions
        """
        img = cv2.imread(str(image_path))
        if img is None:
            print(f"Warning: Could not load {image_path}")
            return
        
        base_name = Path(image_path).stem
        print(f"Processing {base_name}...")
        
        # Save original
        cv2.imwrite(str(self.output_dir / f"{base_name}_original.jpg"), img)
        
        # Generate variations
        for i in range(num_variations):
            augmented = img.copy()
            aug_name = f"{base_name}_aug_{i:03d}"
            
            # Random rotation (-180 to 180 degrees)
            if np.random.random() > 0.3:
                angle = np.random.randint(-180, 180)
                augmented = self.rotate_image(augmented, angle)
                aug_name += f"_rot{angle}"
            
            # Random scale (0.5 to 1.5)
            if np.random.random() > 0.3:
                scale = np.random.uniform(0.5, 1.5)
                augmented = self.scale_image(augmented, scale)
                aug_name += f"_scl{scale:.2f}"
            
            # Crop to content after rotation/scaling
            augmented = self.crop_to_content(augmented, margin=20)
            
            # Save augmented image
            output_path = self.output_dir / f"{aug_name}.jpg"
            cv2.imwrite(str(output_path), augmented)
        
        print(f"  Generated {num_variations} variations for {base_name}")
        
        print(f"\nAugmentation complete! Images saved to: {self.output_dir}")
        total_images = len(list(self.output_dir.glob("*.jpg")))
        print(f"Total images: {total_images}")

In [18]:
# Initialize augmentation pipeline
augmenter = makeSingularCrabImage(output_dir='augmented_crabs')

# List of your original crab images
image_files = ['greenCrab.jpg', 'jonahCrab.jpeg', 'rockCrab.jpg']

# Check if files exist
existing_files = [f for f in image_files if os.path.exists("crabImages/"+f)]

if not existing_files:
    print("Error: No crab images found!")
    print("Please ensure 'greenCrab.jpg', 'jonahCrab.jpg', 'rockCrab.jpg' are in the current directory.")

In [19]:
augmenter.generate_augmented_dataset('crabImages/greenCrab.jpg', num_variations=10)

Processing greenCrab...
  Generated 10 variations for greenCrab

Augmentation complete! Images saved to: augmented_crabs
Total images: 11


In [20]:
augmenter.generate_augmented_dataset('crabImages/jonahCrab.jpeg', num_variations=10)

Processing jonahCrab...
  Generated 10 variations for jonahCrab

Augmentation complete! Images saved to: augmented_crabs
Total images: 22


In [21]:
augmenter.generate_augmented_dataset('crabImages/rockCrab.jpg', num_variations=10)

Processing rockCrab...
  Generated 10 variations for rockCrab

Augmentation complete! Images saved to: augmented_crabs
Total images: 33
